# Automated ML

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.widgets import RunDetails

import joblib
from azureml.core.model import Model

from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice

from train import clean_data

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-gridstability-automl'

experiment=Experiment(ws, experiment_name)

In [3]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-187289
aml-quickstarts-187289
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [4]:
run = experiment.start_logging()

In [5]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ds = pd.read_csv("data/gridstability.csv")

In [7]:
x, y = clean_data(ds)

In [8]:
ds_clean= pd.concat([x, y], axis=1)
ds_clean.to_csv("data/ds_clean.csv", index=False)

In [9]:
datastore = ws.get_default_datastore()

In [10]:
datastore.upload(src_dir='data', target_path='data')

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 4 files
Target already exists. Skipping upload for data/.amlignore
Target already exists. Skipping upload for data/.amlignore.amltmp
Target already exists. Skipping upload for data/ds_clean.csv
Target already exists. Skipping upload for data/gridstability.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_6bb34d491bac405c9ee7258406b36fdf

In [11]:
ds = Dataset.Tabular.from_delimited_files(path=[(datastore, "data/ds_clean.csv")])

In [12]:
ds.take(5).to_pandas_dataframe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,1
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,0
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,1
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,1
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
project_folder = 'gridstability-runs'

In [36]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "blocked_models": ['XGBoostClassifier'],
    "n_cross_validations": 4,
    "enable_dnn": True
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="stabf",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [37]:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on my-cluster with default configuration
Running on remote compute: my-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-gridstability-automl,AutoML_655a796e-f767-46b1-bf8d-7b9ca3f5c0b5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [38]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [39]:
best_run, fit_model = remote_run.get_output()

In [40]:
best_run.get_metrics()

{'log_loss': 0.14876149152505236,
 'matthews_correlation': 0.891962501918334,
 'f1_score_macro': 0.9458787154195882,
 'recall_score_micro': 0.9503,
 'recall_score_weighted': 0.9503,
 'AUC_macro': 0.9905927061411939,
 'norm_macro_recall': 0.8868839164116901,
 'precision_score_micro': 0.9503,
 'AUC_weighted': 0.990592706141194,
 'f1_score_micro': 0.9503,
 'average_precision_score_weighted': 0.9909030242057598,
 'precision_score_weighted': 0.9502047599188647,
 'precision_score_macro': 0.9485358544560608,
 'AUC_micro': 0.9912070000000001,
 'f1_score_weighted': 0.9501573469251874,
 'balanced_accuracy': 0.943441958205845,
 'average_precision_score_macro': 0.989385729782573,
 'recall_score_macro': 0.943441958205845,
 'average_precision_score_micro': 0.9915008418724953,
 'accuracy': 0.9503,
 'weighted_accuracy': 0.956185812516314,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_655a796e-f767-46b1-bf8d-7b9ca3f5c0b5_52/accuracy_table',
 'confusion_matrix': 'aml://artifactId/Experi

In [41]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-gridstability-automl\',\'compute_target\':\'my-cluster\',\'subscription_id\':\'3d1a56d2-7c81-4118-9790-f85d1acf0c77\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_655a796e-f767-46b1-bf8d-7b9ca3f5c0b5_52","experiment_name":"capstone-gridstability-automl","workspace_name":"quick-starts-ws-187289","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group_name":"aml-quickstarts-187289"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '52',
 '_aml_system_scenario_identification': 'Remote.Child

In [42]:
best_run.get_details()

{'runId': 'AutoML_655a796e-f767-46b1-bf8d-7b9ca3f5c0b5_52',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-22T14:33:52.283133Z',
 'endTimeUtc': '2022-02-22T14:33:55.893364Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-gridstability-automl\',\'compute_target\':\'my-cluster\',\'subscription_id\':\'3d1a56d2-7c81-4118-9790-f85d1acf0c77\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_655a796e-f767-46b1-bf8d-7b9ca3f5c0b5_52","experiment_name":"capstone-gridstability-automl","workspace_name":"quick-starts-ws-187289"

In [43]:
# download the environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'environment.yml')

In [57]:
best_run.download_file('outputs/model.pkl', 'best_automl_model.pkl')

In [50]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-gridstability-automl,AutoML_655a796e-f767-46b1-bf8d-7b9ca3f5c0b5_52,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [51]:
joblib.dump(fit_model, filename="outputs/best_model_automl.pkl")

['outputs/best_model_automl.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [90]:
model_name = best_run.properties['model_name']

In [91]:
print(model_name)

AutoML655a796ef52


In [92]:
model = remote_run.register_model(model_name = model_name)

In [93]:
service_name = 'gridstability-api'
script_file_name = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)
best_run_env = best_run.get_environment()
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run_env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-22 15:32:49+00:00 Creating Container Registry if not exists..
2022-02-22 15:42:49+00:00 Registering the environment.
2022-02-22 15:42:50+00:00 Use the existing image..
2022-02-22 15:42:53+00:00 Submitting deployment to compute.
2022-02-22 15:42:57+00:00 Checking the status of deployment gridstability-api..
2022-02-22 15:45:58+00:00 Checking the status of inference endpoint gridstability-api.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [94]:
%run endpoint.py

{"result": [1]}


In [95]:
print(service.get_logs())

2022-02-22T15:45:51,226169200+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-02-22T15:45:51,233173700+00:00 - rsyslog/run 
2022-02-22T15:45:51,248467900+00:00 - iot-server/run 
2022-02-22T15:45:51,273698400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_6797cf9b513e59b405ce80f3e9222a7d/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-22T15:45:51,736885500+00:00 - iot-server/finish 1 0
2022-02-22T15:45:51,760262600+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-02-22 15:45:54,721 | root | INFO | Starting up app insights client
logging socket was

In [ ]:
#service.delete()